In [36]:
'''
Input : Epsilon, Length of qubit, gate decomposition file.
Output : Index of eigenvalues, eigenvalues sorted, average entropy in a text file.

'''

#!/usr/bin/env python
# coding: utf-8

# In[32]:

import sys
from qiskit import*
from qiskit import Aer
import qiskit.quantum_info as qi
import numpy as np
import re
from scipy.sparse import identity
import numpy as np
from scipy import sparse
from scipy.sparse import lil_matrix
import scipy.sparse.linalg
#import csv
#import time
#start = time.time()

   #int(sys.argv[1])
Target_state = '00000000'      #sys.argv[2]



# ## Setting up the The Grover operator
# $$ G = U_w U_s$$
# Where 
# $$U_w = 2|w><w| - I$$ 
# $U_w$ is a matrix with the position of the target state 1 and all its
# diagnoal elements are -1 and rest all zero.

# In[34]:


# First we note the length of N.
N = len(Target_state)


## The operator U_s.
A = np.ones((2**N, 2**N))
U_s = (2/(2**N))*A - np.identity(2**N, dtype = complex)


## The operator U_w. This is neeed for the sign adjustment of Grover_reconstructed operator.
U_w = - np.identity(2 ** N, dtype=complex) 
Target_index = int(Target_state, 2)
U_w.itemset((Target_index, Target_index),1)


## G is the Grover operator.
G = np.matrix(np.matmul(U_w, U_s))




# The following list has all the gates in the format [name of the gate, angle, qubit].
l = []

file1 = open('gates_list_'+Target_state+'.txt', 'r')
Lines = file1.readlines()
 

for line in Lines:
    l.append(line.strip())

gates_list = []


for i in range(len(l)):
    
    l_temp = []
    gate_name = l[i].split(',')[0]
    gate_angle = l[i].split(',')[1]
    gate_qubit = l[i].split(',')[2]
    
    l_temp.append(gate_name)
    l_temp.append(gate_angle)
    l_temp.append(gate_qubit)

    gates_list.append(l_temp)

# ## The basis gates
# The following returns the matrix of the Hadamard, CNOT and RZ gate.

# ### Hadamard gate

# In[39]:


'''

                The Hadamard gate 

'''
def Hadamard_gate(): # Hadamad gate acting on one qubit.
    
    return 1/np.sqrt(2)*np.array([[1,1],[1,-1]])

def RY(theta):
    return np.array([[np.cos(theta/2), -np.sin(theta/2)], [np.sin(theta/2), np.cos(theta/2)]])

def PauliZ():
    return np.array([[1,0],[0,-1]])

# H = RY(pi/2)*Z

def Hadamard(Angle, Qubit): 

    '''

    List below will hold gates acting on one qubit. For example, for L = 3,
    the Hadamard gate acting on the qubit 1 is given by = 1 x H x 1, where 
    x is the Kronecker product. Then, qubits_list = [1,H,1].

    ''' 

    qubits_list = [] 
    
    for i in range(N):
        
        if i == Qubit: # Qubit^th position in the list is H.
            
            qubits_list.append(np.matmul(RY(Angle),PauliZ()))
            
        else: # Other gates are identity operators.
            
            qubits_list.append(np.identity(2))

    '''
    
    The following loop performs the Kronecker product.

    '''        
    
    M = sparse.csr_matrix(qubits_list[0]) # Initializes the final matrix.
    
    for g in range(1,len(qubits_list)):
        
        M = sparse.kron(qubits_list[g],M) # kronecker product.
        
    return M



# ### CNOT gate

# In[40]:


## The dimension of the matrix is fixed by the number of qubits.
def CNOT_qiskit(t,c):
    ## Changing the simulator 
    backend = Aer.get_backend('unitary_simulator')

    ## The circuit without measurement
    circ = QuantumCircuit(N)
    circ.cx(t,c)

    ## job execution and getting the result as an object
    job = execute(circ, backend)
    result = job.result()

    ## get the unitary matrix from the result object
    return result.get_unitary(circ) 


'''
Qiskit uses little endian notation, where the arrangement of the qubits are reversed.
The Kronecker product for the CNOT gate is modified according to the qiskit notation.
Counting starts from 0 and goes to N-1.

'''
def CNOT(c,t,theta):
    
    '''
    Creating the matrix PI0 (|0><0|) and PI1 (|1><1|).
    
    '''
    I = np.identity(2)
    Z = np.matrix([[1,0],[0,-1]])
    X = np.matrix([[0,1],[1,0]])
    PI_0 = (I+Z)/2
    PI_1 = (I-Z)/2
    
    '''
    The following function returns the X gate for theta = pi. Any deviation from pi will
    result in a slightly different gate, which is used to model the noisy X gate.
    
    '''
    def Rx(theta):
        return np.around((np.cos(theta/2)*I-1j*X*np.sin(theta/2))*(np.cos(theta/2)*I+1j*I*np.sin(theta/2)),6)
    
    
    Matrices = {'I':I,'PI_0':PI_0,'X':Rx(theta), 'PI_1':PI_1}
    
    
    '''
    
    We will first create two lists p0 and p1 (for PI0 and PI1) with the matrices
    of the Kronecker product of PI0 and PI1.
    
    '''
    p0 = ['I']*N
    p1 = ['I']*N
    
    
    '''
    The string will be modified according to the position of the target and the control qubits.
    
    '''
        
    p0[c] = 'PI_0'
    p1[c] = 'PI_1'
    p1[t] = 'X'

    

    '''  
    Initialize the PI0 and PI1 matrices as the first elemenst of the list p0 and p1,
    then the following loop will perform the Kronecker product.
    
    '''    
    
    
    
    PI_0_matrix = Matrices[p0[0]]
    for i in range(1,N):
        PI_0_matrix = sparse.kron(Matrices[p0[i]],PI_0_matrix)
        
    PI_1_matrix = Matrices[p1[0]]
    for i in range(1,N):
        PI_1_matrix = sparse.kron(Matrices[p1[i]],PI_1_matrix)

    return PI_0_matrix+PI_1_matrix
# ### RZ gate

# In[41]:


def Rz_matrix(theta):

    return np.matrix([[np.exp(-1j*theta/2),0],[0,np.exp(1j*theta/2)]])

def Rz(Angle, Qubit):
    
    if Qubit > N -1 :
        
        print("Qubit number exceeds N")
        
    else:    
    
        qubits_list = []
    
        for i in range(N):
        
            if i == Qubit:
            
                qubits_list.append(Rz_matrix(Angle))
            
            else:
            
                qubits_list.append(np.matrix(np.identity(2)))
    
        M = sparse.csr_matrix(qubits_list[0])
    
        for g in range(1,len(qubits_list)):
        
            M = sparse.kron(qubits_list[g], M) # kronecker product.
        
        return M        

# ## Adding noise to the Oracle

 



#Rz_Noise = 2*(np.random.rand(Rz_Number)-0.5)



def Grover_reconstructed(epsilon):
    

    #Rz_Noise = 2*(np.random.rand(Rz_Number)-0.5)
    ## Initializing the oracle U_w as an identity matrix.
    
    Or = identity(2**N) 

    ## In the following loop we multiply all the 1 and 2 qubit gates with (or without) noise.
    
    
    for i in range(len(gates_list)): # l is the list with all the gates.
    
        if gates_list[i][0] == 'rz':
            
            Rz_s = Rz(float(gates_list[i][1])  +

                 epsilon * Rz_Noise[i], int(gates_list[i][2]))
            
            Or = Or*Rz_s
            
        
        
        
        elif gates_list[i][0] == 'h':
            
            
            Hs = Hadamard(np.pi/2+epsilon*Rz_Noise[i], int(gates_list[i][2]))
            Or = Or*Hs
            
        
        elif gates_list[i][0] == 'cx':

            Or = Or*CNOT(int(gates_list[i][1]), int(gates_list[i][2]),np.pi+epsilon*Rz_Noise[i])
     
    #Or = Or.todense()
    ## In the following we will fix the phase of the reconstructed Oracle.
    # First we will make all the elements
    # 1 or -1.
    #Or = Or/Or[0,0]
    
    ## The sign of the reconstructed Oracle should be same as that of original U_w.
    #if np.sign(Or[0,0]) == np.sign(U_w[0,0]):
        
        #pass # If the sign is same, then pass.
    
    #else:
        
        #Or = -Or # Otherwise change the sign.
    Gr = np.matmul(Or, U_s) ## The Grover operator G = U_w * U_s.
    
    return Gr

In [30]:
np.random.seed(2020)
Rz_Noise = 2*(np.random.rand(len(gates_list))-0.5)

In [31]:
from numpy import linalg as LA

In [32]:
def Normalized(Array):
    return Array/LA.norm(Array)

def Pxbar(Array):
    Array_list = np.array(Psi).ravel()
    s = 0.0
    for i in Array_list[1:]:
        s += i
    return s*np.conjugate(s)/(2**N-1)


In [33]:
# Initial wave function.
def Psi_0(N):
    psi_array = 1/(np.sqrt(2**N))*np.ones(2**N)
    return np.matrix(psi_array)

In [43]:
Op = np.matrix(Grover_reconstructed(0.0))

ValueError: matmul: Input operand 0 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

In [42]:
Op

matrix([[-9.80693891e-01+6.53759758e-04j,
         -6.68548374e-01-7.87867480e-01j,
          9.95840371e-03+3.76863991e-01j, ...,
          1.93061093e-02+6.53759758e-04j,
          1.93061093e-02+6.53759758e-04j,
          1.93061093e-02+6.53759758e-04j],
        [ 1.17270023e-01+8.38384144e-01j,
          5.87265386e-01-4.79842160e-01j,
         -3.66690299e-01+4.35988304e-01j, ...,
         -9.96450782e-03-1.11729589e-02j,
         -9.96450782e-03-1.11729589e-02j,
         -9.96450782e-03-1.11729589e-02j],
        [-2.77091413e-01+3.16823727e-01j,
          4.30840329e-01-4.72512571e-01j,
          1.06609846e+00-7.28974534e-01j, ...,
         -7.26563258e-03+1.23078362e-02j,
         -7.26563258e-03+1.23078362e-02j,
         -7.26563258e-03+1.23078362e-02j],
        ...,
        [ 4.98955953e-03-1.84587413e-03j,
          4.98955953e-03-1.84587413e-03j,
          4.98955953e-03-1.84587413e-03j, ...,
         -4.36983366e-02-9.31478486e-01j,
         -7.36191192e-01+2.29190830e-01j

In [38]:
f = open('probability_data.txt','w')
Psi = Psi_0(N)
Psi = Psi.transpose()
p0 = []
pxbar = []

U = Op
for i in range(10):
    f = open('probability_data.txt','a')
    if i == 0:
        p1 = Psi[0,0]*np.conjugate(Psi[0,0])
        p2 = Pxbar(Psi[1:])
        f.write(str(p1.real) +'\t'+str(p2.real)+'\t'+str(i)+'\n')
    else:
        Psi = np.matmul(U,Psi)
        #Psi = Normalized(Psi)
        Psi = np.array(Psi).ravel()
        p1 = Psi[0]*np.conjugate(Psi[0])
        p2 = Pxbar(Psi[1:])
        f.write(str(p1.real) +'\t'+str(p2.real)+'\t'+str(i)+'\n')
        
    p0.append(p1)
    pxbar.append(p2)    
    f.close()

In [39]:
p0

[0.00390625,
 (0.02388180857053964+0j),
 (0.13998238533156138+0j),
 (0.21516459956546327+0j),
 (0.8865734418400232+0j),
 (1.1018476885928932+0j),
 (1.7491172986011523+0j),
 (0.8947949284737975+0j),
 (2.719394816712035+0j),
 (37.32791653676327+0j)]

In [40]:
pxbar

[0.99609375,
 (0.26094040041874544+0j),
 (0.45548076137123483+0j),
 (0.6858428547263838+0j),
 (3.025400659338582+0j),
 (6.174179809003642+0j),
 (18.66877599786361+0j),
 (34.671595557037485+0j),
 (96.7674712703065+0j),
 (239.67960143468846+0j)]

In [ ]:
Psi = Normalized(Psi)
Psi = Psi/LA.norm(Psi)
LA.norm(Psi)